In [1]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

Using TensorFlow backend.


### Домашнее задание 5

#### 1.Попробуйте изменить параметры нейронной сети работающей с датасетом imdb так, чтобы улучшить ее точность. Приложите анализ.

In [2]:
max_features = 20000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 80
batch_size = 200 # увеличьте значение для ускорения обучения

print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности


In [3]:
print('Pad последовательности (примеров в x единицу времени)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [4]:
import keras

In [5]:
keras.layers.LSTM?

In [6]:
opt = keras.optimizers.Adam(learning_rate=0.001,
                            amsgrad=True,
                            )

In [7]:
print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 256))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров 
model.compile(loss='binary_crossentropy',
              
              optimizer=opt,
              metrics=['accuracy'])


print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=4, # увеличьте при необходимости
          validation_data=(x_test, y_test))

score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)


Построение модели...
Процесс обучения...


C:\Users\Pavel\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 17s 670us/step - loss: 0.4906 - accuracy: 0.7542 - val_loss: 0.3774 - val_accuracy: 0.8334
Epoch 2/4
25000/25000 [==============================] - 15s 614us/step - loss: 0.2898 - accuracy: 0.8836 - val_loss: 0.3735 - val_accuracy: 0.8329
Epoch 3/4
25000/25000 [==============================] - 15s 611us/step - loss: 0.2189 - accuracy: 0.9183 - val_loss: 0.4034 - val_accuracy: 0.8316
Epoch 4/4
25000/25000 [==============================] - 3s 135us/step
Результат при тестировании: 0.44444111108779905
Тестовая точность: 0.824720025062561


Начальная точность - 0.8392 (с batch size = 200) <br>
Каких-то улучший с помощью изменения оптимизатора добиться не получилось <br>
Увеличение количества слов до 100 снизило результат до 0.8146 <br>
Изменение количества юнитов в рамках от 64 до 256 не оказало существенного влияния <br>
Добавление Dense слоя не изменило картину. <br>
С увеличением количества эпох точность на валиде не растет <br>
<b>Какого-то внятного вывода сделать пока не получается. Модель не так чувствительна к подбору параметров, как из предыдущих курсов. Вероятно, это связанно с самим принципом работы реккурентных сетей
</b>

#### Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения.

In [8]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential


# построчное чтение из примера с текстом 
with open("Устав.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("UTF-8", "strict")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)


# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 256
NUM_ITERATIONS = 12 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 6
NUM_PREDS_PER_EPOCH = 150


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. 
За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы
в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию
ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    LSTM(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
Epoch 1/6
415557/415557 [==============================] - 24s 57us/step - loss: 2.0430
Epoch 2/6
415557/415557 [==============================] - 23s 56us/step - loss: 1.3260
Epoch 3/6
415557/415557 [==============================] - 23s 56us/step - loss: 1.1212
Epoch 4/6
415557/415557 [==============================] - 23s 56us/step - loss: 1.0103
Epoch 5/6
415557/415557 [==============================] - 24s 58us/step - loss: 0.9371
Epoch 6/6
415557/415557 [==============================] - 24s 57us/step - loss: 0.8824
Генерация из посева: енней служ
енней службы в полку (подразделений и старшего и старшине роты. В полку и должности приема пищи должен приведения в полку (подразделений и старшего и старшине р==================================================
Итерация #: 1
Epoch 1/6
415557/415557 [==============================] - 24s 57us/step - loss: 0.8390
Epoch 2/6
415557/415557 [==============================] - 23s 56us/step - loss: 0.8024
Epoch 3/6
415557/415557 [

415557/415557 [==============================] - 23s 55us/step - loss: 0.3980
Epoch 5/6
415557/415557 [==============================] - 23s 55us/step - loss: 0.3971
Epoch 6/6
415557/415557 [==============================] - 24s 58us/step - loss: 0.3958
Генерация из посева: лужбы в ба
лужбы в батальоне мероприкатра вооружения и военной службы и соблюдать при размещения подразделениями полка (корабля) и его подразделениями полка (корабля) и ег==================================================
Итерация #: 11
Epoch 1/6
415557/415557 [==============================] - 23s 56us/step - loss: 0.3950
Epoch 2/6
415557/415557 [==============================] - 23s 55us/step - loss: 0.3943
Epoch 3/6
415557/415557 [==============================] - 23s 55us/step - loss: 0.3930
Epoch 4/6
415557/415557 [==============================] - 23s 55us/step - loss: 0.3921
Epoch 5/6
415557/415557 [==============================] - 23s 55us/step - loss: 0.3907
Epoch 6/6
415557/415557 [=======================

Устав был выбран как потенциально простой текст, лишенный художественных приемов <br> 
Попробовал поиграться с функциями активации, кол-вом юнитов, размерами батча и остальными доступными вариантами. <br>
Результат всегда практически одинаковый: мы получаем в среднем бессмысленное предложение. Не очень понятно, в чем проблема: в архитектуре такой сети, либо в размере и качестве датасета. <br>
** Дополнительные задания поставили в тупик, на данном этапе знакомства с нейросетями - слишком хардкорно.